# Plot vegetation coverage prediction using PointNet-based model
## Dataset

The working dataset contains ~200 circulat plots of 10m radius.
Each point has 9 features : XYZ, RGB, NIR, intensity, return number

<table><tr>
<td> <img src="exemples_images/POINT_OBS9_cl.png" alt="Drawing" style="width: 250px;"/> </td>
<td> <img src="exemples_images/POINT_OBS10_cl.png" alt="Drawing" style="width: 250px;"/> </td>
</tr></table>


Let's first start by importing and installing the necessary libraires:

In [39]:
import warnings
warnings.simplefilter(action='ignore')

import functools
import argparse
import numpy as np
import pandas as pd

from sklearn.model_selection import KFold
from torch.utils.tensorboard import SummaryWriter
from scipy.stats import gamma
import os
import time
import torch
import torch.optim as optim
from torch.optim.lr_scheduler import StepLR
import torchnet as tnt
from sklearn.neighbors import NearestNeighbors

import gc
from torch_scatter import scatter_max, scatter_mean

import torch.nn as nn
from laspy.file import File
from sklearn.neighbors import NearestNeighbors
from scipy.special import digamma, polygamma
import matplotlib
import matplotlib.pyplot as plt
matplotlib.use('TkAgg')

import pickle

print(torch.cuda.is_available())
np.random.seed(42)
torch.cuda.empty_cache()

True


We set model parameters:

In [40]:
parser = argparse.ArgumentParser(description='model')


# System Parameters
parser.add_argument('--path', default="/home/ign.fr/ekalinicheva/DATASET_regression/", type=str,
                    help="Main folder directory")
parser.add_argument('--gt_file', default="resultats_placettes_combo.csv", type=str, help="Name of GT *.cvs file")
parser.add_argument('--plot_folder_name', default="placettes_combo", type=str, help="Name of GT *.csv file")
parser.add_argument('--cuda', default=1, type=int, help="Whether we use cuda (1) or not (0)")
parser.add_argument('--folds', default=5, type=int, help="Number of folds for cross validation model training")

# Model Parameters
parser.add_argument('--n_class', default=4, type=int,
                    help="Size of the model output vector. In our case 4 - different vegetation coverage types")
parser.add_argument('--input_feats', default='xyzrgbnir', type=str,
                    help="Point features that we keep. in this code, we keep them all. permuting those letters will break everything. To be modified")
parser.add_argument('--subsample_size', default=4096, type=int, help="Subsample cloud size")
parser.add_argument('--diam_pix', default=32, type=int,
                    help="Size of the output stratum raster (its diameter in pixels)")
parser.add_argument('--m', default=1, type=float,
                    help="Loss regularization. The weight of the negative loglikelihood loss in the total loss")
parser.add_argument('--norm_ground', default=False, type=bool,
                    help="Whether we normalize low vegetation and bare soil values, so LV+BS=1 (True) or we keep unmodified LV value (False) (recommended)")
parser.add_argument('--adm', default=False, type=bool, help="Whether we compute admissibility or not")
parser.add_argument('--nb_stratum', default=3, type=int,
                    help="[2, 3] Number of vegetation stratum that we compute 2 - ground level + medium level; 3 - ground level + medium level + high level")
parser.add_argument('--ECM_ite_max', default=5, type=int, help='Max number of EVM iteration')
parser.add_argument('--NR_ite_max', default=10, type=int, help='Max number of Netwon-Rachson iteration')

# Network Parameters
parser.add_argument('--MLP_1', default=[32, 32], type=list,
                    help="Parameters of the 1st MLP block (output size of each layer). See PointNet article")
parser.add_argument('--MLP_2', default=[64, 128], type=list,
                    help="Parameters of the 2nd MLP block (output size of each layer). See PointNet article")
parser.add_argument('--MLP_3', default=[64, 32], type=list,
                    help="Parameters of the 3rd MLP block (output size of each layer). See PointNet article")
parser.add_argument('--drop', default=0.4, type=float, help="Probability value of the DropOut layer of the model")
parser.add_argument('--soft', default=True, type=bool,
                    help="Whether we use softmax layer for the model output (True) of sigmoid (False)")

# Optimization Parameters
parser.add_argument('--wd', default=0.001, type=float, help="Weight decay for the optimizer")
parser.add_argument('--lr', default=1e-3, type=float, help="Learning rate")
parser.add_argument('--step_size', default=50, type=int,
                    help="After this number of steps we decrease learning rate. (Period of learning rate decay)")
parser.add_argument('--lr_decay', default=0.1, type=float,
                    help="We multiply learning rate by this value after certain number of steps (see --step_size). (Multiplicative factor of learning rate decay)")
parser.add_argument('--n_epoch', default=150, type=int, help="Number of training epochs")
parser.add_argument('--n_epoch_test', default=5, type=int, help="We evaluate every -th epoch")
parser.add_argument('--batch_size', default=20, type=int, help="Size of the training batch")

args, _ = parser.parse_known_args()


assert (args.nb_stratum in [2, 3]), "Number of stratum should be 2 or 3!"
assert (args.lr_decay < 1), "Learning rate decrease should be smaller than 1, as learning rate should decrease"

Print stats to file

In [41]:
def print_stats(stats_file, text, print_to_console=True):
    with open(stats_file, 'a') as f:
        if isinstance(text, list):
            for t in text:
                f.write(t + "\n")
                if print_to_console:
                    print(t)
        else:
            f.write(text + "\n")
            if print_to_console:
                print(text)
    f.close()

Function to create a new folder if does not exists

In [42]:
def create_dir(dir_name):
    if not os.path.exists(dir_name):
        os.makedirs(dir_name)

We define some paths to files and folders and check starting time

In [43]:
las_folder = os.path.join(args.path, args.plot_folder_name) # folder with las files

# We keep track of time and stats
start_time = time.time()
print(time.strftime("%H:%M:%S", time.gmtime(start_time)))
run_name = str(time.strftime("%Y-%m-%d_%H%M%S"))

# We write results to different folders depending on the chosen parameters
if args.nb_stratum == 2:
    results_path = stats_path = os.path.join(args.path, "RESULTS/")
else:
    results_path = stats_path = os.path.join(args.path,"RESULTS_3_stratum/")

if args.adm:
    results_path = os.path.join(results_path, "admissibility/")
else:
    results_path = os.path.join(results_path, "only_stratum/")

stats_path = os.path.join(results_path, run_name) + "/"
print("Results folder: ", stats_path)
stats_file = os.path.join(stats_path, "stats.txt")
create_dir(stats_path)


16:57:40
Results folder:  /home/ign.fr/ekalinicheva/DATASET_regression/RESULTS_3_stratum/only_stratum/2021-05-06_185740/


We open las files with plots as numpy arrays:

In [44]:
def open_las(las_folder):
    # We open las files and create a training dataset
    dataset = {}  # dict to store numpy array with each plot separately
    mean_dataset = {}  # we keep track of plots means to reverse the normalisation in the future

    # We iterate through las files and transform them to np array
    las_files = os.listdir(las_folder)
    all_points = np.empty((0, 9))
    for las_file in las_files:
        las = File(os.path.join(las_folder, las_file), mode='r')
        x_las = las.X
        y_las = las.Y
        z_las = las.Z
        r = las.Red
        g = las.Green
        b = las.Blue
        nir = las.nir
        intensity = las.intensity
        nbr_returns = las.return_num
        points_placette = np.asarray([x_las / 100, y_las / 100, z_las / 100, r, g, b, nir, intensity,
                                      nbr_returns]).T  # we divide by 100 as all the values in las are in cm

        # There is a file with 2 points 60m above others (maybe birds), we delete these points
        if las_file == "Releve_Lidar_F70.las":
            points_placette = points_placette[points_placette[:, 2] < 640]
        # We do the same for the intensity
        if las_file == "POINT_OBS8.las":
            points_placette = points_placette[points_placette[:, -2] < 32768]
        if las_file == "Releve_Lidar_F39.las":
            points_placette = points_placette[points_placette[:, -2] < 20000]

        # We directly substract z_min at local level
        xyz = points_placette[:, :3]
        knn = NearestNeighbors(500, algorithm='kd_tree').fit(xyz[:, :2])
        _, neigh = knn.radius_neighbors(xyz[:, :2], 0.5)
        z = xyz[:, 2]
        zmin_neigh = []
        for n in range(len(z)):
            zmin_neigh.append(np.min(z[neigh[n]]))
        points_placette[:, 2] = points_placette[:, 2] - zmin_neigh

        all_points = np.append(all_points, points_placette, axis=0)
        dataset[os.path.splitext(las_file)[0]] = points_placette
        mean_dataset[os.path.splitext(las_file)[0]] = [np.mean(x_las) / 100, np.mean(y_las) / 100]

    return all_points, dataset, mean_dataset

In [45]:
print("Loading data in memory")
all_points, dataset, mean_dataset = open_las(las_folder)
print("Our dataset contains " + str(len(dataset)) + " plots!" )

Loading data in memory
Our dataset contains 209 plots!


We extract last parameters and write all of them to stats file:

In [46]:
z_all = all_points[:, 2]
args.z_max = np.max(z_all)   # maximum z value for data normalization, obtained from the normalized dataset analysis
args.n_input_feats = len(args.input_feats)  # number of input features
print_stats(stats_file, str(args), print_to_console=True)  # save all the args parameters

Namespace(ECM_ite_max=5, MLP_1=[32, 32], MLP_2=[64, 128], MLP_3=[64, 32], NR_ite_max=10, adm=False, batch_size=20, cuda=1, diam_pix=32, drop=0.4, folds=5, gt_file='resultats_placettes_combo.csv', input_feats='xyzrgbnir', lr=0.001, lr_decay=0.1, m=1, n_class=4, n_epoch=1, n_epoch_test=1, n_input_feats=9, nb_stratum=3, norm_ground=False, path='/home/ign.fr/ekalinicheva/DATASET_regression/', plot_folder_name='placettes_combo', soft=True, step_size=50, subsample_size=4096, wd=0.001, z_max=24.140000000000043)


We compute parameters of gamma distributions for two stratum

In [47]:
def get_gamma_parameters(all_z, args):
    all_z = all_z + 1e-2
    #initialization
    shape = np.array([0.2, 1.8])  # scale gamma parameters
    scale = np.array([0.3, 4]) #shape gamma parameters
    pi = np.array([0.5, 5]) #bernoulli parameter
    def view_distribution():
        fig, ax = plt.subplots(1, 1)
        x = np.linspace(0,10, 100)
        plt.hist(all_z, bins=100, range=(0, 10), density=True)
        plt.plot(x, pi[0] * gamma.pdf(x, shape[0], 0, scale[0]), 'r-', lw=1, label='gamma1')
        plt.plot(x, pi[1] * gamma.pdf(x, shape[1], 0, scale[1]), 'k-', lw=1, label='gamma2')
        plt.tight_layout()
        axes = plt.gca()
        axes.set_ylim([0,0.5])
        plt.show(block=True)
    def E_step():
        expected_values = np.vstack((gamma.pdf(all_z, shape[0], 0, scale[0]),gamma.pdf(all_z, shape[1], 0, scale[1])))
        expected_values = expected_values * pi[:, None]
        return expected_values/expected_values.sum(0)
    def inner_optim(expected_values):
        #find simultaneously the CM values for shape defined as poles obj, with with newton-raphson
        x = shape
        def obj(x):
            #the function to minimize
            return (expected_values * (np.log(all_z[None,:]) - np.log(scale)[:,None] - digamma(x)[:,None])).mean(1)
        def derivative(x):
            #its derivative
            return (expected_values * (- polygamma(1, x)[:, None])).mean(1)
        for sub_ite in range(args.NR_ite_max):
            print("    NR it %d - obj = %3.3f %3.3f" % (sub_ite, *obj(x)))
            if (np.abs(obj(x))<1e-3).all():
                print("Newton Rachson terminated")
                break
            x = x - obj(x) / derivative(x) #one NR iteration
        return x
    def CM1(expected_values):
        #first CM-step
        pi = expected_values.mean(1)
        scale = inner_optim(expected_values)
        return pi, scale
    def CM2(expected_values):
        #second CM-step
        num = (all_z[None,:] * expected_values).mean(1)
        denom = scale * expected_values.mean(1)
        return num/denom
    def log_likelihood():
        expected_values = np.vstack((gamma.pdf(all_z, shape[0], 0, scale[0]), gamma.pdf(all_z, shape[1], 0, scale[1])))
        expected_values = expected_values * pi[:, None]
        return -np.log(expected_values.sum(0)).mean()
    #---main loop---
    print("Likelihood at init: %2.3f" % (log_likelihood()))
    for ite in range(args.ECM_ite_max):
        expected_values = E_step()
        pi, scale = CM1(expected_values)
        shape = CM2(expected_values)
        print("Likelihood at ite %d: %2.3f" % (ite, log_likelihood()))

    # We plot the distributions
    x = np.linspace(0, 10, 101)[1:]
    histo = plt.hist(all_z, bins=100, range=(0, 10), density=True)
    bins = histo[1][1:]
    pdf =  histo[0]
    y1 = pi[0] * gamma.pdf(x, shape[0], 0, scale[0])
    y2 = pi[1] * gamma.pdf(x, shape[1], 0, scale[1])
    # np.savetxt("ECM.csv",  np.vstack((bins, pdf, y1, y2)).transpose(), delimiter=",")
    view_distribution()

    params = {'phi': pi[0], 'a_g': shape[0], 'a_v': shape[1],
              'loc_g': 0, 'loc_v': 0, 'scale_g': scale[0],
              'scale_v': scale[1]}
    return params

In [48]:
gamma_file = os.path.join(stats_path, "gamma.pkl")
if not os.path.isfile(gamma_file):
    print("Computing gamma mixture (should only happen once)")
    params = get_gamma_parameters(z_all, args)
    with open(gamma_file, 'wb') as f:
        pickle.dump(params, f)
else:
    print("Found precomputed Gamma parameters")
    with open(gamma_file, 'rb') as f:
        params = pickle.load(f)
print_stats(stats_file, str(params), print_to_console=True)

Computing gamma mixture (should only happen once)
Likelihood at init: 0.425
    NR it 0 - obj = 1.785 -0.284
    NR it 1 - obj = 0.703 0.119
    NR it 2 - obj = 0.194 0.011
    NR it 3 - obj = 0.023 0.000
    NR it 4 - obj = 0.000 0.000
Newton Rachson terminated
Likelihood at ite 0: 1.653
    NR it 0 - obj = 3.083 0.038
    NR it 1 - obj = 1.282 0.002
    NR it 2 - obj = 0.412 0.000
    NR it 3 - obj = 0.070 0.000
    NR it 4 - obj = 0.003 0.000
    NR it 5 - obj = 0.000 -0.000
Newton Rachson terminated
Likelihood at ite 1: 1.423
    NR it 0 - obj = 1.225 0.030
    NR it 1 - obj = 0.400 0.001
    NR it 2 - obj = 0.071 0.000
    NR it 3 - obj = 0.003 0.000
    NR it 4 - obj = 0.000 0.000
Newton Rachson terminated
Likelihood at ite 2: 1.446
    NR it 0 - obj = 1.639 0.009
    NR it 1 - obj = 0.584 0.000
    NR it 2 - obj = 0.128 0.000
    NR it 3 - obj = 0.009 0.000
    NR it 4 - obj = 0.000 0.000
Newton Rachson terminated
Likelihood at ite 3: 1.410
    NR it 0 - obj = 1.155 0.030
    NR

## Datalaoder


In [49]:
def augment(cloud_data):
    """augmentation function
    Does random rotation around z axis and adds Gaussian noise to all the features, except z and return number
    """
    #random rotation around the Z axis
    #angle = random angle 0..2pi
    angle = np.radians(np.random.choice(360, 1)[0])
    c, s = np.cos(angle), np.sin(angle)
    M = np.array(((c, -s), (s, c))) #rotation matrix around axis z with angle "angle"
    cloud_data[:2] = np.dot(cloud_data[:2].T, M).T #perform the rotation efficiently


    #random gaussian noise everywhere except z and return number
    sigma, clip = 0.01, 0.03
    cloud_data[:2] = cloud_data[:2] + np.clip(sigma*np.random.randn(cloud_data[:2].shape[0], cloud_data[:2].shape[1]), a_min=-clip, a_max=clip).astype(np.float32)
    cloud_data[3:8] = cloud_data[3:8] + np.clip(sigma*np.random.randn(cloud_data[3:8].shape[0], cloud_data[3:8].shape[1]), a_min=-clip, a_max=clip).astype(np.float32)
    return cloud_data


def cloud_loader(plot_id, dataset, df_gt, train, args):
    """
    load a plot and returns points features (normalized xyz + features) and
    ground truth
    INPUT:
    tile_name = string, name of the tile
    train = int, train = 1 iff in the train set
    OUTPUT
    cloud_data, [n x 4] float Tensor containing points coordinates and intensity
    labels, [n] long int Tensor, containing the points semantic labels
    """
    cloud_data = np.array(dataset[plot_id]).transpose()
    gt = df_gt[df_gt['Name']==plot_id][['COUV_BASSE', 'COUV_SOL', 'COUV_INTER', 'COUV_HAUTE', 'ADM']].values/100
    # gt = np.asarray([np.append(gt, [1 - gt[:, 2]])])

    xmean, ymean = np.mean(cloud_data[0:2], axis=1)

    #normalizing data
    # Z data was already partially normalized during loading
    cloud_data[0] = (cloud_data[0] - xmean)/10 #x
    cloud_data[1] = (cloud_data[1] - ymean)/10 #y
    cloud_data[2] = (cloud_data[2])/args.z_max #z


    colors_max = 65536
    cloud_data[3:7] = cloud_data[3:7]/colors_max
    int_max = 32768
    cloud_data[7] = cloud_data[7] / int_max
    cloud_data[8] = (cloud_data[8] - 1)/(7-1)

    if train:
      cloud_data = augment(cloud_data)

    cloud_data = torch.from_numpy(cloud_data)
    gt = torch.from_numpy(gt).float()
    return cloud_data, gt


def cloud_collate(batch):
    """ Collates a list of dataset samples into a batch list for clouds
    and a single array for labels
    This function is necessary to implement because the clouds have different sizes (unlike for images)
    """
    clouds, labels = list(zip(*batch))
    labels = torch.cat(labels, 0)
    return clouds, labels

Functions to create final images

In [50]:
plt.rcParams["font.size"] = 25


def visualize_article(image_soil, image_med_veg, image_high_veg, cloud, pl_id, stats_path, args, txt=None):

    fig = plt.figure(figsize=(15, 12))
    gs = gridspec.GridSpec(3, 3)

    # Original point data
    ax1 = fig.add_subplot(gs[:, 0:2], projection='3d')
    colors_ = cloud[3:6].numpy().transpose()
    ax1.scatter(cloud[0], cloud[1], cloud[2]*args.z_max, c=colors_, vmin=0, vmax=1, s=10, alpha=1)
    ax1.auto_scale_xyz
    ax1.set_yticklabels([])
    ax1.set_xticklabels([])
    # colors = cloud[3:7].numpy().transpose()
    # ax1.scatter3D(cloud[0], cloud[1], cloud[2], c=cloud[[6, 3, 4]].numpy().transpose(), s=2, vmin=0, vmax=10)
    ax1.set_title(pl_id)
    for line in ax1.xaxis.get_ticklines():
        line.set_visible(False)
    for line in ax1.yaxis.get_ticklines():
        line.set_visible(False)


    # LV stratum raster
    ax2 = fig.add_subplot(gs[0, 2])
    color_grad = [(0.8, 0.4, 0.1), (0, 1, 0)]  # first color is white, last is green
    cm = colors.LinearSegmentedColormap.from_list(
        "Custom", color_grad, N=100)
    ax2.imshow(image_soil, cmap=cm, vmin=0, vmax=1)
    ax2.set_title('Ground level')
    ax2.tick_params(
        axis='both',  # changes apply to both axis
        which='both',  # both major and minor ticks are affected
        bottom=False,  # ticks along the bottom edge are off
        top=False,  # ticks along the top edge are off
        left=False,
        right=False,
        labelbottom=False)  # labels along the bottom edge are off
    ax2.set_yticklabels([])
    ax2.set_xticklabels([])


    # MV stratum raster
    ax3 = fig.add_subplot(gs[1, 2])
    color_grad = [(1, 1, 1), (0, 1, 0)]  # first color is white, last is green
    cm = colors.LinearSegmentedColormap.from_list(
        "Custom", color_grad, N=100)
    ax3.imshow(image_med_veg, cmap=cm, vmin=0, vmax=1)
    ax3.set_title("Medium level")
    ax3.tick_params(
        axis='both',  # changes apply to the x-axis
        which='both',  # both major and minor ticks are affected
        bottom=False,  # ticks along the bottom edge are off
        top=False,  # ticks along the top edge are off
        left=False,
        right=False,
        labelbottom=False)  # labels along the bottom edge are off
    ax3.set_yticklabels([])
    ax3.set_xticklabels([])


    # Plot high vegetation stratum
    ax4 = fig.add_subplot(gs[2, 2])
    color_grad = [(1, 1, 1), (0, 1, 0)]  # first color is white, last is red
    cm = colors.LinearSegmentedColormap.from_list(
        "Custom", color_grad, N=100)
    ax4.imshow(image_high_veg, cmap=cm, vmin=0, vmax=1)
    ax4.set_title("High level")
    ax4.tick_params(
        axis='both',  # changes apply to the x-axis
        which='both',  # both major and minor ticks are affected
        bottom=False,  # ticks along the bottom edge are off
        top=False,  # ticks along the top edge are off
        left=False,
        right=False,
        labelbottom=False)  # labels along the bottom edge are off
    ax4.set_yticklabels([])
    ax4.set_xticklabels([])

    if txt is not None:
        fig.text(.5, .05, txt, ha='center')
    plt.savefig(stats_path + pl_id + '_article.svg', format="svg", bbox_inches="tight", dpi=300)


def visualize(image_soil, image_med_veg, cloud, prediction, pl_id, stats_path, args, txt=None, scores=None, image_high_veg=None):

    if image_soil.ndim==3:
        image_soil = image_soil[:,:,0]
        image_med_veg = image_med_veg[:, :, 0]


    # We set figure size depending on the number of subplots
    if scores is None and image_high_veg is None:
        row, col = 2, 2
        fig = plt.figure(figsize=(20, 15))
    else:
        row, col = 3, 2
        fig = plt.figure(figsize=(20, 25))

    # Original point data
    ax1 = fig.add_subplot(row, col, 1, projection='3d')
    colors_ = cloud[3:6].numpy().transpose()
    ax1.scatter(cloud[0], cloud[1], cloud[2]*args.z_max, c=colors_, vmin=0, vmax=1, s=10, alpha=1)
    ax1.auto_scale_xyz
    ax1.set_yticklabels([])
    ax1.set_xticklabels([])
    # colors = cloud[3:7].numpy().transpose()
    # ax1.scatter3D(cloud[0], cloud[1], cloud[2], c=cloud[[6, 3, 4]].numpy().transpose(), s=2, vmin=0, vmax=10)
    ax1.set_title(pl_id)


    # LV stratum raster
    ax2 = fig.add_subplot(row, col, 2)
    color_grad = [(0.8, 0.4, 0.1), (0, 1, 0)]  # first color is brown, last is green
    cm = colors.LinearSegmentedColormap.from_list(
        "Custom", color_grad, N=100)
    ax2.imshow(image_soil, cmap=cm, vmin=0, vmax=1)
    ax2.set_title('Ground coverage')
    ax2.tick_params(
        axis='both',  # changes apply to both axis
        which='both',  # both major and minor ticks are affected
        bottom=False,  # ticks along the bottom edge are off
        top=False,  # ticks along the top edge are off
        left=False,
        right=False,
        labelbottom=False)  # labels along the bottom edge are off
    ax2.set_yticklabels([])
    ax2.set_xticklabels([])


    # Pointwise prediction
    ax3 = fig.add_subplot(row, col, 3, projection='3d')
    ax3.auto_scale_xyz
    colors_pred = prediction.cpu().detach().numpy().transpose()
    color_matrix = [[0, 1, 0],
                    [0.8, 0.4, 0.1],
                    [0, 0, 1],
                    [1, 0, 0]]
    colors_pred = np.matmul(colors_pred, color_matrix)
    ax3.scatter(cloud[0], cloud[1], cloud[2]*args.z_max, c=colors_pred, s=10, vmin=0, vmax=1, alpha=1)
    ax3.set_title('Pointwise prediction')
    ax3.set_yticklabels([])
    ax3.set_xticklabels([])


    # MV stratum raster
    ax4 = fig.add_subplot(row, col, 4)
    color_grad = [(1, 1, 1), (0, 0, 1)]  # first color is white, last is blue
    cm = colors.LinearSegmentedColormap.from_list(
        "Custom", color_grad, N=100)
    ax4.imshow(image_med_veg, cmap=cm, vmin=0, vmax=1)
    ax4.set_title("Medium vegetation coverage")
    ax4.tick_params(
        axis='both',  # changes apply to the x-axis
        which='both',  # both major and minor ticks are affected
        bottom=False,  # ticks along the bottom edge are off
        top=False,  # ticks along the top edge are off
        left=False,
        right=False,
        labelbottom=False)  # labels along the bottom edge are off
    ax4.set_yticklabels([])
    ax4.set_xticklabels([])

    # Plot stratum scores
    if scores is not None:
        ax5 = fig.add_subplot(row, col, 5, projection='3d')
        ax5.auto_scale_xyz
        sc_sum = scores.sum(1)
        scores[:, 0] = scores[:, 0] / sc_sum
        scores[:, 1] = scores[:, 1] / sc_sum
        scores = scores/(scores.max())
        colors_pred = scores.transpose(1, 0)[[0, 0, 1], :].cpu().detach().numpy().transpose()
        ax5.scatter(cloud[0], cloud[1], cloud[2] * args.z_max, c=colors_pred, s=10, vmin=0, vmax=1)
        ax5.set_title("Strate scores")
        ax5.set_yticklabels([])
        ax5.set_xticklabels([])


    # Plot high vegetation stratum
    if image_high_veg is not None:
        ax6 = fig.add_subplot(row, col, 6)
        color_grad = [(1, 1, 1), (1, 0, 0)]  # first color is white, last is red
        cm = colors.LinearSegmentedColormap.from_list(
            "Custom", color_grad, N=100)
        ax6.imshow(image_high_veg, cmap=cm, vmin=0, vmax=1)
        ax6.set_title("High vegetation coverage")
        ax6.tick_params(
            axis='both',  # changes apply to the x-axis
            which='both',  # both major and minor ticks are affected
            bottom=False,  # ticks along the bottom edge are off
            top=False,  # ticks along the top edge are off
            left=False,
            right=False,
            labelbottom=False)  # labels along the bottom edge are off
        ax6.set_yticklabels([])
        ax6.set_xticklabels([])

    if txt is not None:
        fig.text(.5, .05, txt, ha='center')
    plt.savefig(stats_path + pl_id + '.png', format="png", bbox_inches="tight", dpi=300)



def create_final_images(pred_pl, gt, pred_pointwise_b, cloud, likelihood, plot_name, mean_dataset, stats_path,
                        stats_file, args, create_raster=True, adm=None):
    '''
    We do final data reprojection to the 2D space (2 stratum - ground vegetation level and medium level, optionally high level) by associating the points to the pixels.
    Then we create the images with those stratum
    '''
    for b in range(len(pred_pointwise_b)):
        # we get prediction stats string
        pred_cloud = pred_pointwise_b[b]
        current_cloud = cloud[b]
        # we do raster reprojection, but we do not use torch scatter as we have to associate each value to a pixel
        xy = current_cloud[:2]
        xy = torch.floor((xy - torch.min(xy, dim=1).values.view(2, 1).expand_as(xy)) / (
                torch.max(xy, dim=1).values - torch.min(xy, dim=1).values + 0.0001).view(2, 1).expand_as(
            xy) * args.diam_pix).int()
        xy = xy.cpu().numpy()
        unique, index, inverse = np.unique(xy.T, axis=0, return_index=True, return_inverse=True)

        # we get the values for each unique pixel and write them to rasters
        image_ground = np.full((args.diam_pix, args.diam_pix), np.nan)
        image_med_veg = np.full((args.diam_pix, args.diam_pix), np.nan)
        if args.nb_stratum == 3:
            image_high_veg = np.full((args.diam_pix, args.diam_pix), np.nan)
        else:
            image_high_veg = None
        for i in np.unique(inverse):
            where = np.where(inverse == i)[0]
            k, m = xy.T[where][0]
            maxpool = nn.MaxPool1d(len(where))
            max_pool_val = maxpool(pred_cloud[:, where].unsqueeze(0)).cpu().detach().numpy().flatten()

            if args.norm_ground:  # we normalize ground level coverage values
                proba_low_veg = max_pool_val[0] / (max_pool_val[:2].sum())
            else:   # we do not normalize anything, as bare soil coverage does not participate in absolute loss
                proba_low_veg = max_pool_val[0]
            proba_med_veg = max_pool_val[2]
            image_ground[m, k] = proba_low_veg
            image_med_veg[m, k] = proba_med_veg

            if args.nb_stratum == 3:
                proba_high_veg = max_pool_val[3]
                image_high_veg[m, k] = proba_high_veg
        image_ground = np.flip(image_ground, axis=0)  # we flip along y axis as the 1st raster row starts with 0
        image_med_veg = np.flip(image_med_veg, axis=0)
        if args.nb_stratum == 3:
            image_high_veg = np.flip(image_high_veg, axis=0)
        # We normalize back x,y values to create a tiff file with 2 rasters
        if create_raster:
            xy = xy * 10 + np.asarray(mean_dataset[plot_name]).reshape(-1, 1)
            geo = [np.min(xy, axis=1)[0], (np.max(xy, axis=1)[0] - np.min(xy, axis=1)[0]) / args.diam_pix, 0,
                   np.max(xy, axis=1)[1], 0, (-np.max(xy, axis=1)[1] + np.min(xy, axis=1)[1]) / args.diam_pix]
            if args.nb_stratum == 2:
                img_to_write = np.concatenate(([image_ground], [image_med_veg]), 0)
            else:
                img_to_write = np.concatenate(([image_ground], [image_med_veg], [image_high_veg]), 0)
            create_tiff(nb_channels=args.nb_stratum, new_tiff_name=stats_path + plot_name + ".tif", width=args.diam_pix,
                        height=args.diam_pix, datatype=gdal.GDT_Float32, data_array=img_to_write, geotransformation=geo)
        if args.adm:
            text = 'Pred ' + np.array2string(
                np.round(np.asarray(pred_pl[b].cpu().detach().numpy().reshape(-1)), 2)) + ' ADM ' + str(
                adm[b].cpu().detach().numpy().round(2)) + ' GT ' + np.array2string(
                gt.cpu().numpy()[0])  # prediction text
        else:
            text = ' Pred ' + np.array2string(
                np.round(np.asarray(pred_pl[b].cpu().detach().numpy().reshape(-1)), 2)) + ' GT ' + np.array2string(
                gt.cpu().numpy()[0])
        print_stats(stats_file, plot_name + " " + text, print_to_console=True)
        # We create an image with 5 or 6 subplots:
        # 1. original point cloud, 2. LV image, 3. pointwise prediction point cloud, 4. MV image, 5.Stratum probabilities point cloud, 6.(optional) HV image
        visualize(image_ground, image_med_veg, current_cloud, pred_cloud, plot_name, stats_path, args, txt=text,
                  scores=likelihood, image_high_veg=image_high_veg)
        if args.nb_stratum==3:
            visualize_article(image_ground, image_med_veg, image_high_veg, current_cloud, plot_name, stats_path, args, txt=text)


# We create a tiff file with 2 or 3 stratum
def create_tiff(nb_channels, new_tiff_name, width, height, datatype, data_array, geotransformation):
    # We set Lambert 93 projection
    srs = osr.SpatialReference()
    srs.ImportFromEPSG(2154)
    proj = srs.ExportToWkt()
    # We create a datasource
    driver_tiff = gdal.GetDriverByName("GTiff")
    dst_ds = driver_tiff.Create(new_tiff_name, width, height, nb_channels, datatype)
    if nb_channels == 1:
        dst_ds.GetRasterBand(1).WriteArray(data_array)
    else:
        for ch in range(nb_channels):
            dst_ds.GetRasterBand(ch + 1).WriteArray(data_array[ch])
    dst_ds.SetGeoTransform(geotransformation)
    dst_ds.SetProjection(proj)
    return dst_ds


## PointNet model for point classification

In [51]:
class PointNet(nn.Module):
    """
    The PointNet network for semantic segmentation
    """

    def __init__(self, MLP_1, MLP_2, MLP_3, args):
        """
        initialization function
        MLP_1, LMP2 and MLP3 = int array, size of the layers of multi-layer perceptrons
        for example MLP1 = [32,64]
        n_class = int,  the number of class
        input_feat = int, number of input feature
        subsample_size = int, number of points to which the tiles are subsampled

        """

        super(PointNet, self).__init__()  # necessary for all classes extending the module class

        self.is_cuda = args.cuda
        self.subsample_size = args.subsample_size
        self.n_class = args.n_class
        self.drop = args.drop
        self.soft = args.soft
        self.input_feat = args.n_input_feats

        # since we don't know the number of layers in the MLPs, we need to use loops
        # to create the correct number of layers
        m1 = MLP_1[-1]  # size of the first embeding F1
        m2 = MLP_2[-1]  # size of the second embeding F2

        # build MLP_1: input [input_feat x n] -> f1 [m1 x n]
        modules = []
        for i in range(len(MLP_1)):  # loop over the layer of MLP1
            # note: for the first layer, the first in_channels is feature_size
            modules.append(
                nn.Conv1d(in_channels=MLP_1[i - 1] if i > 0 else self.input_feat, out_channels=MLP_1[i], kernel_size=1))
            modules.append(nn.BatchNorm1d(MLP_1[i]))
            modules.append(nn.ReLU(True))
        # this transform the list of layers into a callable module
        self.MLP_1 = nn.Sequential(*modules)

        # build MLP_2: f1 [m1 x n] -> f2 [m2 x n]
        modules = []
        for i in range(len(MLP_2)):
            modules.append(nn.Conv1d(in_channels=MLP_2[i - 1] if i > 0 else m1, out_channels=MLP_2[i], kernel_size=1))
            modules.append(nn.BatchNorm1d(MLP_2[i]))
            modules.append(nn.ReLU(True))
        self.MLP_2 = nn.Sequential(*modules)

        # build MLP_3: f1 [(m1 + m2) x n] -> output [k x n]
        modules = []
        for i in range(len(MLP_3)):
            modules.append(
                nn.Conv1d(in_channels=MLP_3[i - 1] if i > 0 else (m1 + m2), out_channels=MLP_3[i], kernel_size=1))
            modules.append(nn.BatchNorm1d(MLP_3[i]))
            modules.append(nn.ReLU(True))
        # note: the last layer do not have normalization nor activation
        modules.append(nn.Dropout(p=self.drop))
        modules.append(nn.Conv1d(MLP_3[-1], self.n_class, 1))
        self.MLP_3 = nn.Sequential(*modules)

        self.maxpool = nn.MaxPool1d(self.subsample_size)
        self.softmax = nn.Softmax(dim=1)
        self.sigmoid = nn.Sigmoid()

        if self.is_cuda:
            self = self.cuda()

    def forward(self, input):
        """
        the forward function producing the embeddings for each point of 'input'
        input = [n_batch, input_feat, subsample_size] float array: input features
        output = [n_batch,n_class, subsample_size] float array: point class logits
        """
        # print(input.size())
        if self.is_cuda:
            input = input.cuda()
        f1 = self.MLP_1(input)
        f2 = self.MLP_2(f1)
        G = self.maxpool(f2)
        Gf1 = torch.cat((G.repeat(1, 1, self.subsample_size), f1), 1)
        out_pointwise = self.MLP_3(Gf1)
        if self.soft:
            out_pointwise = self.softmax(out_pointwise)
        else:
            out_pointwise = self.sigmoid(out_pointwise)
        return out_pointwise

Different functions to keep track of statistics:
    - Per epoch
    - Per Fold
    - Mean for the whole dataset

In [52]:
# We compute all possible mean stats per loss for all folds
def stats_for_all_folds(all_folds_loss_train_lists, all_folds_loss_test_lists, stats_file, args):
    loss_train_list, loss_train_abs_list, loss_train_log_list, loss_train_adm_list = all_folds_loss_train_lists
    loss_test_list, loss_test_abs_list, loss_test_log_list, loss_test_abs_gl_list, loss_test_abs_ml_list, loss_test_abs_hl_list, loss_test_adm_list = all_folds_loss_test_lists


    if args.adm:
        mean_cross_fold_train = np.mean(loss_train_list), np.mean(loss_train_abs_list), np.mean(
            loss_train_log_list), np.mean(loss_train_adm_list)
        print_stats(stats_file,
                    "Mean Train Loss " + str(mean_cross_fold_train[0]) + " Loss abs " + str(
                        mean_cross_fold_train[1]) + " Loss log " + str(mean_cross_fold_train[2]) + " Loss ADM " + str(
                        mean_cross_fold_train[3]),
                    print_to_console=True)

        if args.nb_stratum == 2:
            mean_cross_fold_test = np.mean(loss_test_list), np.mean(loss_test_abs_list), np.mean(
                loss_test_log_list), np.mean(loss_test_abs_gl_list), np.mean(loss_test_abs_ml_list), np.mean(
                loss_test_adm_list)

            print_stats(stats_file,
                        "Mean Test Loss " + str(mean_cross_fold_test[0]) + " Loss abs " + str(
                            mean_cross_fold_test[1]) + " Loss log " + str(
                            mean_cross_fold_test[2]) + " Loss abs GL " + str(
                            mean_cross_fold_test[3]) + " Loss abs ML " + str(
                            mean_cross_fold_test[4]) + " Loss ADM " + str(mean_cross_fold_test[5]),
                        print_to_console=True)

        else:  # 3 stratum
            mean_cross_fold_test = np.mean(loss_test_list), np.mean(loss_test_abs_list), np.mean(
                loss_test_log_list), np.mean(loss_test_abs_gl_list), np.mean(loss_test_abs_ml_list), np.mean(
                loss_test_abs_hl_list), np.mean(loss_test_adm_list)

            print_stats(stats_file,
                        "Mean Test Loss " + str(mean_cross_fold_test[0]) + " Loss abs " + str(
                            mean_cross_fold_test[1]) + " Loss log " + str(
                            mean_cross_fold_test[2]) + " Loss abs GL " + str(
                            mean_cross_fold_test[3]) + " Loss abs ML " + str(
                            mean_cross_fold_test[4]) + " Loss abs HL " + str(
                            mean_cross_fold_test[5]) + " Loss ADM " + str(mean_cross_fold_test[6]),
                        print_to_console=True)


    else:
        mean_cross_fold_train = np.mean(loss_train_list), np.mean(loss_train_abs_list), np.mean(loss_train_log_list)
        print_stats(stats_file,
                    "Mean Train Loss " + str(mean_cross_fold_train[0]) + " Loss abs " + str(
                        mean_cross_fold_train[1]) + " Loss log " + str(mean_cross_fold_train[2]),
                    print_to_console=True)

        if args.nb_stratum == 2:
            mean_cross_fold_test = np.mean(loss_test_list), np.mean(loss_test_abs_list), np.mean(
                loss_test_log_list), np.mean(loss_test_abs_gl_list), np.mean(loss_test_abs_ml_list)

            print_stats(stats_file,
                        "Mean Test Loss " + str(mean_cross_fold_test[0]) + " Loss abs " + str(
                            mean_cross_fold_test[1]) + " Loss log " + str(
                            mean_cross_fold_test[2]) + " Loss abs GL " + str(
                            mean_cross_fold_test[3]) + " Loss abs ML " + str(
                            mean_cross_fold_test[4]),
                        print_to_console=True)

        else:  # 3 stratum
            mean_cross_fold_test = np.mean(loss_test_list), np.mean(loss_test_abs_list), np.mean(
                loss_test_log_list), np.mean(loss_test_abs_gl_list), np.mean(loss_test_abs_ml_list), np.mean(
                loss_test_abs_hl_list)

            print_stats(stats_file,
                        "Mean Test Loss " + str(mean_cross_fold_test[0]) + " Loss abs " + str(
                            mean_cross_fold_test[1]) + " Loss log " + str(
                            mean_cross_fold_test[2]) + " Loss abs GL " + str(
                            mean_cross_fold_test[3]) + " Loss abs ML " + str(
                            mean_cross_fold_test[4]) + " Loss abs HL " + str(
                            mean_cross_fold_test[5]),
                        print_to_console=True)



# We compute all possible loss stats per fold
def stats_per_fold(all_folds_loss_train_lists, all_folds_loss_test_lists, final_train_losses_list, final_test_losses_list, stats_file, fold_id, args):
    if all_folds_loss_test_lists is None and all_folds_loss_test_lists is None:
        # We keep track of stats per fold
        loss_train_list = []
        loss_train_abs_list = []
        loss_train_log_list = []
        loss_train_adm_list = []
        loss_test_list = []
        loss_test_abs_list = []
        loss_test_log_list = []
        loss_test_abs_gl_list = []
        loss_test_abs_ml_list = []
        loss_test_abs_hl_list = []
        loss_test_adm_list = []
    else:
        loss_train_list, loss_train_abs_list, loss_train_log_list, loss_train_adm_list = all_folds_loss_train_lists
        loss_test_list, loss_test_abs_list, loss_test_log_list, loss_test_abs_gl_list, loss_test_abs_ml_list, loss_test_abs_hl_list, loss_test_adm_list = all_folds_loss_test_lists


    loss_train, loss_train_abs, loss_train_log, loss_train_adm = final_train_losses_list
    loss_test, loss_test_abs, loss_test_log, loss_test_abs_gl, loss_test_abs_ml, loss_test_abs_hl, loss_test_adm = final_test_losses_list

    # Save all loss stats
    print_stats(stats_file,
                "Fold_" + str(fold_id) + " Train Loss " + str(loss_train) + " Loss abs " + str(
                    loss_train_abs) + " Loss log " + str(loss_train_log),
                print_to_console=True)
    if args.adm:
        print_stats(stats_file,
                    "Fold_" + str(fold_id) + " Test Loss " + str(loss_test) + " Loss abs " + str(
                        loss_test_abs) + " Loss log " + str(loss_test_log) + " Loss abs adm " + str(loss_test_adm),
                    print_to_console=True)
    else:
        print_stats(stats_file,
                    "Fold_" + str(fold_id) + " Test Loss " + str(loss_test) + " Loss abs " + str(
                        loss_test_abs) + " Loss log " + str(loss_test_log),
                    print_to_console=True)

    if args.nb_stratum == 2:
        print_stats(stats_file,
                    "Fold_" + str(fold_id) + " Test Loss abs GL " + str(loss_test_abs_gl) + " Test Loss abs ML " + str(
                        loss_test_abs_ml), print_to_console=True)
    else:
        print_stats(stats_file,
                    "Fold_" + str(fold_id) + " Test Loss abs GL " + str(loss_test_abs_gl) + " Test Loss abs ML " + str(
                        loss_test_abs_ml) + " Test Loss abs HL " + str(
                        loss_test_abs_hl), print_to_console=True)

    loss_train_list.append(loss_train)
    loss_train_abs_list.append(loss_train_abs)
    loss_train_log_list.append(loss_train_log)
    loss_train_adm_list.append(loss_train_adm)

    loss_test_list.append(loss_test)
    loss_test_abs_list.append(loss_test_abs)
    loss_test_log_list.append(loss_test_log)
    loss_test_abs_gl_list.append(loss_test_abs_gl)
    loss_test_abs_ml_list.append(loss_test_abs_ml)
    loss_test_abs_hl_list.append(loss_test_abs_hl)
    loss_test_adm_list.append(loss_test_adm)

    all_folds_loss_train_lists = [loss_train_list, loss_train_abs_list, loss_train_log_list, loss_train_adm_list]
    all_folds_loss_test_lists = [loss_test_list, loss_test_abs_list, loss_test_log_list, loss_test_abs_gl_list, loss_test_abs_ml_list, loss_test_abs_hl_list, loss_test_adm_list]
    return all_folds_loss_train_lists, all_folds_loss_test_lists



#We perform tensorboard visualisation by writing the stats of each epoch to the writer
def write_to_writer(writer, args, i_epoch, list_with_losses, train):
    TESTCOLOR = '\033[104m'
    TRAINCOLOR = '\033[100m'
    NORMALCOLOR = '\033[0m'

    if train:
        loss_train, loss_train_abs, loss_train_log, loss_train_adm = list_with_losses
        if args.adm:
            print(
                TRAINCOLOR + 'Epoch %3d -> Train Loss: %1.4f Train Loss Abs: %1.4f Train Loss Log: %1.4f Train Loss Adm: %1.4f' % (
                i_epoch, loss_train, loss_train_abs, loss_train_log, loss_train_adm) + NORMALCOLOR)
            writer.add_scalar('Loss/train_abs_adm', loss_train_adm, i_epoch + 1)
        else:
            print(TRAINCOLOR + 'Epoch %3d -> Train Loss: %1.4f Train Loss Abs: %1.4f Train Loss Log: %1.4f' % (
            i_epoch, loss_train, loss_train_abs, loss_train_log) + NORMALCOLOR)
        writer.add_scalar('Loss/train', loss_train, i_epoch + 1)
        writer.add_scalar('Loss/train_abs', loss_train_abs, i_epoch + 1)
        writer.add_scalar('Loss/train_log', loss_train_log, i_epoch + 1)

    else:
        loss_test, loss_test_abs, loss_test_log, _, _, _, loss_test_adm = list_with_losses
        if args.adm:
            print(TESTCOLOR + 'Test Loss: %1.4f Test Loss Abs: %1.4f Test Loss Log: %1.4f Test Loss Adm: %1.4f' % (
            loss_test, loss_test_abs, loss_test_log, loss_test_adm) + NORMALCOLOR)
            writer.add_scalar('Loss/test_abs_adm', loss_test_adm, i_epoch + 1)
        else:
            print(TESTCOLOR + 'Test Loss: %1.4f Test Loss Abs: %1.4f Test Loss Log: %1.4f' % (
                loss_test, loss_test_abs, loss_test_log) + NORMALCOLOR)
        writer.add_scalar('Loss/test', loss_test, i_epoch + 1)
        writer.add_scalar('Loss/test_abs', loss_test_abs, i_epoch + 1)
        writer.add_scalar('Loss/test_log', loss_test_log, i_epoch + 1)
    return writer


Here we perform 3D data projection into different stratum and compute the vegetation ratio of each stratum.

In [53]:
def project_to_2d(pred_pointwise, cloud, pred_pointwise_b, PCC, args):
    """
    We do all the computation to obtain
    pred_pl - [Bx4] prediction vector for the plot
    scores -  [(BxN)x2] probas_ground_nonground that a point belongs to stratum 1 or stratum 2
    """
    index_batches = []
    index_group = []
    batches_len = []

    # we project 3D points to 2D plane
    # We use torch scatter to process
    for b in range(len(pred_pointwise_b)):
        current_cloud = cloud[b]
        xy = current_cloud[:2]
        xy = torch.floor((xy - torch.min(xy, dim=1).values.view(2, 1).expand_as(xy)) / (
                torch.max(xy, dim=1).values - torch.min(xy, dim=1).values + 0.0001).view(2, 1).expand_as(
            xy) * args.diam_pix).int()

        unique, index = torch.unique(xy.T, dim=0, return_inverse=True)
        index_b = torch.full(torch.unique(index).size(), b)
        if PCC.is_cuda:
            index = index.cuda()
            index_b = index_b.cuda()
        index = index + np.asarray(batches_len).sum()
        index_batches.append(index.type(torch.LongTensor))
        index_group.append(index_b.type(torch.LongTensor))
        batches_len.append(torch.unique(index).size(0))
    index_batches = torch.cat(index_batches)
    index_group = torch.cat(index_group)
    if PCC.is_cuda:
        index_batches = index_batches.cuda()
        index_group = index_group.cuda()
    pixel_max = scatter_max(pred_pointwise.T, index_batches)[0]

    # We compute prediction values per pixel
    if args.norm_ground:    # we normalize ground level coverage values, so c_low[i]+c_bare[i]=1
        c_low_veg_pix = pixel_max[0, :] / (pixel_max[:2, :].sum(0))
        c_bare_soil_pix = pixel_max[1, :] / (pixel_max[:2, :].sum(0))
    else:   # we do not normalize anything, as bare soil coverage does not participate in absolute loss
        c_low_veg_pix = pixel_max[0, :]
        c_bare_soil_pix = 1 - c_low_veg_pix
    c_med_veg_pix = pixel_max[2, :]

    if args.nb_stratum==2:
        # We compute prediction values per plot
        c_low_veg = scatter_mean(c_low_veg_pix, index_group)
        c_bare_soil = scatter_mean(c_bare_soil_pix, index_group)
        c_med_veg = scatter_mean(c_med_veg_pix, index_group)
        # c_other = scatter_mean(c_other_pix, index_group)
        pred_pl = torch.stack([c_low_veg, c_bare_soil, c_med_veg]).T

    else:   # 3 stratum
        c_high_veg_pix = pixel_max[3, :]    # we equally compute raster for high vegetation

        # We compute prediction values per plot
        c_low_veg = scatter_mean(c_low_veg_pix, index_group)
        c_bare_soil = scatter_mean(c_bare_soil_pix, index_group)
        c_med_veg = scatter_mean(c_med_veg_pix, index_group)
        c_high_veg = scatter_mean(c_high_veg_pix, index_group)
        pred_pl = torch.stack([c_low_veg, c_bare_soil, c_med_veg, c_high_veg]).T

    if args.adm:
        c_adm_pix = torch.max(pixel_max[[0,2], :], dim=0)[0]
        c_adm = scatter_mean(c_adm_pix, index_group)
    else:
        c_adm = None

    return pred_pl, c_adm




One epoch model training function:

In [54]:
def train(model, PCC, train_set, params, optimizer, args):
    """train for one epoch"""
    model.train()

    # the loader function will take care of the batching
    loader = torch.utils.data.DataLoader(train_set, collate_fn=cloud_collate, \
                                         batch_size=args.batch_size, shuffle=True, drop_last=True)

    # will keep track of the loss
    loss_meter = tnt.meter.AverageValueMeter()
    loss_meter_abs = tnt.meter.AverageValueMeter()
    loss_meter_log = tnt.meter.AverageValueMeter()
    loss_meter_abs_adm = tnt.meter.AverageValueMeter()

    for index_batch, (cloud, gt) in enumerate(loader):

        if PCC.is_cuda:
            gt = gt.cuda()

        optimizer.zero_grad()  # put gradient to zero
        pred_pointwise, pred_pointwise_b = PCC.run(model, cloud)  # compute the pointwise prediction
        pred_pl, pred_adm = project_to_2d(pred_pointwise, cloud, pred_pointwise_b, PCC, args)  # compute plot prediction

        # we compute two losses (negative loglikelihood and the absolute error loss for 2 or 3 stratum)
        loss_abs = loss_absolute(pred_pl, gt, args)
        loss_log, likelihood = loss_loglikelihood(pred_pointwise, cloud, params, PCC,
                                                  args)  # negative loglikelihood loss
        if args.adm:
            # we compute admissibility loss
            loss_adm = loss_abs_adm(pred_adm, gt)
            loss = loss_abs + args.m * loss_log + 0.5 * loss_adm
            loss_meter_abs_adm.add(loss_adm.item())
        else:
            loss = loss_abs + args.m * loss_log

        loss.backward()
        optimizer.step()

        loss_meter_abs.add(loss_abs.item())
        loss_meter_log.add(loss_log.item())
        loss_meter.add(loss.item())
        gc.collect()

    return loss_meter.value()[0], loss_meter_abs.value()[0], loss_meter_log.value()[0], loss_meter_abs_adm.value()[0]


One epoch model evaluation function:

In [55]:

# We import from other files
from utils.create_final_images import *
from data_loader.loader import *
from utils.reproject_to_2d_and_predict_plot_coverage import *
from model.loss_functions import *


np.random.seed(42)


def eval(model, PCC, test_set, params, args, test_list, mean_dataset, stats_path, stats_file, last_epoch=False):
    """eval on test set"""

    model.eval()

    loader = torch.utils.data.DataLoader(test_set, collate_fn=cloud_collate, batch_size=1, shuffle=False)
    loss_meter_abs = tnt.meter.AverageValueMeter()
    loss_meter_log = tnt.meter.AverageValueMeter()
    loss_meter = tnt.meter.AverageValueMeter()
    loss_meter_abs_gl = tnt.meter.AverageValueMeter()
    loss_meter_abs_ml = tnt.meter.AverageValueMeter()
    loss_meter_abs_hl = tnt.meter.AverageValueMeter()
    loss_meter_abs_adm = tnt.meter.AverageValueMeter()


    for index_batch, (cloud, gt) in enumerate(loader):

        if PCC.is_cuda:
            gt = gt.cuda()


        start_encoding_time = time.time()
        pred_pointwise, pred_pointwise_b = PCC.run(model, cloud)  # compute the prediction
        end_encoding_time = time.time()
        if last_epoch:            # if it is the last epoch, we get time stats info
            print(end_encoding_time - start_encoding_time)


        pred_pl, pred_adm = project_to_2d(pred_pointwise, cloud, pred_pointwise_b, PCC, args) # compute plot prediction

        # we compute two losses (negative loglikelihood and the absolute error loss for 2 stratum)
        loss_abs = loss_absolute(pred_pl, gt, args)  # absolut loss
        loss_log, likelihood = loss_loglikelihood(pred_pointwise, cloud, params, PCC,
                                                  args)  # negative loglikelihood loss
        if args.adm:
            # we compute admissibility loss
            loss_adm = loss_abs_adm(pred_adm, gt)
            loss = loss_abs + args.m * loss_log + 0.5 * loss_adm
            loss_meter_abs_adm.add(loss_adm.item())
        else:
            loss = loss_abs + args.m * loss_log

        loss_meter.add(loss.item())
        loss_meter_abs.add(loss_abs.item())
        loss_meter_log.add(loss_log.item())
        gc.collect()

        if last_epoch:

            component_losses = loss_absolute(pred_pl, gt, args, level_loss=True) # gl_mv_loss gives separated losses for each stratum
            if args.nb_stratum == 2:
                loss_abs_gl, loss_abs_ml = component_losses
            else:
                loss_abs_gl, loss_abs_ml, loss_abs_hl = component_losses
                loss_abs_hl = loss_abs_hl[~torch.isnan(loss_abs_hl)]
                if loss_abs_hl.size(0) > 0:
                    loss_meter_abs_hl.add(loss_abs_hl.item())
            loss_meter_abs_gl.add(loss_abs_gl.item())
            loss_meter_abs_ml.add(loss_abs_ml.item())

            create_final_images(pred_pl, gt, pred_pointwise_b, cloud, likelihood, test_list[index_batch], mean_dataset, stats_path, stats_file,
                                args, adm=pred_adm)  # create final images with stratum values


    return loss_meter.value()[0], loss_meter_abs.value()[0], loss_meter_log.value()[0], loss_meter_abs_gl.value()[0], loss_meter_abs_ml.value()[0], loss_meter_abs_hl.value()[0], loss_meter_abs_adm.value()[0]


For efficiency concerns, we want all the point clouds we consider to have the same number of points `subsample_size`. This allows for easy GPU parallelization. As the tiles are of different sizes, we sample a fixed number of points from each tile (with replacement). The labels of the sampled points are then spatially interpolated to the original point cloud using a nearest neighbor strategy.
`PointCloudClassifier` class takes care of sampling, batching, prediciton, and interpolation of differently-sized clouds.


In [56]:
class PointCloudClassifier:
    """
    The main point cloud classifier Class
    deal with subsampling the tiles to a fixed number of points
    and interpolating to the original clouds
    """

    def __init__(self, args):
        self.subsample_size = args.subsample_size  # number of points to subsample each point cloud in the batches
        self.n_input_feats = 3  # size of the point descriptors in input
        if len(args.input_feats) > 3:
            self.n_input_feats = len(args.input_feats)
        self.n_class = args.n_class  # number of classes in the output
        self.is_cuda = args.cuda  # wether to use GPU acceleration

    def run(self, model, clouds):
        """
        INPUT:
        model = the neural network
        clouds = list of n_batch tensors of size [n_feat, n_points_i]: batch of point clouds
        OUTPUT:
        pred = [sum_i n_points_i, n_class] float tensor : prediction for each element of the
             batch in a single tensor

        """

        # number of clouds in the batch #TYPO
        n_batch = len(clouds)
        # will contain the prediction for all clouds in the batch
        prediction_batch = torch.zeros((self.n_class, 0))

        # batch_data contain all the clouds in the batch subsampled to self.subsample_size points
        sampled_clouds = torch.Tensor(n_batch, self.n_input_feats, self.subsample_size)
        if self.is_cuda:
            sampled_clouds = sampled_clouds.cuda()
            prediction_batch = prediction_batch.cuda()

        # build batches of the same size
        for i_batch in range(n_batch):
            # load the elements in the batch one by one and subsample/ oversample them
            # to a size of self.subsample_size points

            cloud = clouds[i_batch][:int(self.n_input_feats), :]
            n_points = cloud.shape[1]  # number of points in the considered cloud
            if n_points > self.subsample_size:
                selected_points = np.random.choice(n_points, self.subsample_size,
                                                   replace=False)
            else:
                selected_points = np.random.choice(n_points, self.subsample_size,
                                                   replace=True)
            cloud = cloud[:, selected_points]  # reduce the current cloud to the selected points

            sampled_clouds[i_batch, :, :] = cloud.clone()  # place current sampled cloud in sampled_clouds

        point_prediction = model(sampled_clouds)  # classify the batch of sampled clouds
        assert (point_prediction.shape == torch.Size([n_batch, self.n_class, self.subsample_size]))

        # interpolation to original point clouds
        prediction_batches = []
        for i_batch in range(n_batch):
            # get the original point clouds positions
            cloud = clouds[i_batch]
            # and the corresponding sampled batch (only xyz position)
            sampled_cloud = sampled_clouds[i_batch, :3, :]
            n_points = cloud.shape[1]
            knn = NearestNeighbors(1, algorithm='kd_tree').fit( \
                sampled_cloud.cpu().permute(1, 0))
            # select for each point in the original point cloud the closest point in sampled_cloud
            _, closest_point = knn.kneighbors(cloud[:3, :].permute(1, 0).cpu())
            closest_point = closest_point.squeeze()
            prediction_cloud = point_prediction[i_batch, :, closest_point]
            prediction_batch = torch.cat((prediction_batch, prediction_cloud), 1)
            prediction_batches.append(prediction_cloud)
        return prediction_batch.permute(1, 0), prediction_batches

We train the model during `n_epoch` epochs and evaluate it every i-th (`n_epoch_test`) epoch.

In [57]:
def train_full(args, fold_id):
    """The full training loop"""
    # initialize the model
    model = PointNet(args.MLP_1, args.MLP_2, args.MLP_3, args)
    writer = SummaryWriter(results_path + "runs/"+run_name + "fold_" + str(fold_id) +"/")

    print('Total number of parameters: {}'.format(sum([p.numel() for p in model.parameters()])))
    print(model)

    # define the classifier
    PCC = PointCloudClassifier(args)

    # define the optimizer
    optimizer = optim.Adam(model.parameters(), lr=args.lr, weight_decay=args.wd)
    scheduler = StepLR(optimizer, step_size=args.step_size, gamma=args.lr_decay)

    for i_epoch in range(args.n_epoch):
        scheduler.step()

        # train one epoch
        train_losses = train(model, PCC, train_set, params, optimizer, args)
        writer = write_to_writer(writer, args, i_epoch, train_losses, train=True)

        if (i_epoch + 1) % args.n_epoch_test == 0:
            if (i_epoch + 1) == args.n_epoch:   # if last epoch, we creare 2D images with points projections
                test_losses = eval(model, PCC, test_set, params, args, test_list, mean_dataset, stats_path, stats_file, last_epoch=True)
            else:
                test_losses = eval(model, PCC, test_set, params, args, test_list, mean_dataset, stats_path, stats_file)
            gc.collect()
            writer = write_to_writer(writer, args, i_epoch, test_losses, train=False)
    writer.flush()

    final_train_losses_list = train_losses
    final_test_losses_list = test_losses
    return model, final_train_losses_list, final_test_losses_list


We finally train the model for `args.fold` folds and do the cross validation:

In [58]:
df_gt = pd.read_csv(os.path.join(args.path, args.gt_file), sep=',', header=0)  # we open GT file
placettes = df_gt['Name'].to_numpy()    # We extract the names of the plots to create train and test list

# We use several folds for cross validation (set the number in args)
kf = KFold(n_splits=args.folds, random_state=42, shuffle=True)

# None lists that will stock stats for each fold, so we can compute the mean at the end
all_folds_loss_train_lists = None
all_folds_loss_test_lists = None

#cross-validation
fold_id = 1
print("Starting cross-validation")
for train_ind, test_ind in kf.split(placettes):
    print("Cross-validation FOLD = %d" % (fold_id))
    train_list = placettes[train_ind]
    test_list = placettes[test_ind]

    # generate the train and test dataset
    test_set = tnt.dataset.ListDataset(test_list,
                                       functools.partial(cloud_loader, dataset=dataset, df_gt=df_gt, train=False, args=args))
    train_set = tnt.dataset.ListDataset(train_list,
                                        functools.partial(cloud_loader, dataset=dataset, df_gt=df_gt, train=True, args=args))

    trained_model, final_train_losses_list, final_test_losses_list = train_full(args, fold_id)

    # save the trained model
    PATH = os.path.join(stats_path, "model_ss_" + str(args.subsample_size) + "_dp_" + str(args.diam_pix) + "_fold_" + str(fold_id) + ".pt")
    torch.save(trained_model, PATH)

    # We compute stats per fold
    all_folds_loss_train_lists, all_folds_loss_test_lists = stats_per_fold(all_folds_loss_train_lists, all_folds_loss_test_lists, final_train_losses_list, final_test_losses_list, stats_file, fold_id, args)

    print_stats(stats_file,
                "training time " + str(time.strftime("%H:%M:%S", time.gmtime(time.time() - start_time))),
                print_to_console=True)
    fold_id += 1

Starting cross-validation
Cross-validation FOLD = 1
Total number of parameters: 25028
PointNet(
  (MLP_1): Sequential(
    (0): Conv1d(9, 32, kernel_size=(1,), stride=(1,))
    (1): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): Conv1d(32, 32, kernel_size=(1,), stride=(1,))
    (4): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace=True)
  )
  (MLP_2): Sequential(
    (0): Conv1d(32, 64, kernel_size=(1,), stride=(1,))
    (1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): Conv1d(64, 128, kernel_size=(1,), stride=(1,))
    (4): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace=True)
  )
  (MLP_3): Sequential(
    (0): Conv1d(160, 64, kernel_size=(1,), stride=(1,))
    (1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats

Exception in Tkinter callback
Traceback (most recent call last):
  File "/home/ign.fr/ekalinicheva/anaconda3/envs/safe_environment/lib/python3.7/tkinter/__init__.py", line 1705, in __call__
    return self.func(*args)
  File "/home/ign.fr/ekalinicheva/anaconda3/envs/safe_environment/lib/python3.7/tkinter/__init__.py", line 749, in callit
    func(*args)
  File "/home/ign.fr/ekalinicheva/anaconda3/envs/safe_environment/lib/python3.7/site-packages/matplotlib/backends/_backend_tk.py", line 270, in idle_draw
    self.draw()
  File "/home/ign.fr/ekalinicheva/anaconda3/envs/safe_environment/lib/python3.7/site-packages/matplotlib/backends/backend_tkagg.py", line 9, in draw
    super(FigureCanvasTkAgg, self).draw()
  File "/home/ign.fr/ekalinicheva/anaconda3/envs/safe_environment/lib/python3.7/site-packages/matplotlib/backends/backend_agg.py", line 393, in draw
    self.figure.draw(self.renderer)
  File "/home/ign.fr/ekalinicheva/anaconda3/envs/safe_environment/lib/python3.7/site-packages/matp

KeyboardInterrupt: 

Compute mean stats for 5 folds

In [ ]:
stats_for_all_folds(all_folds_loss_train_lists, all_folds_loss_test_lists, stats_file, args)